In [ ]:
'''装饰器'''
# 返回一个函数 可能是原函数对象 也可能是内部重新定义的
# 装饰器的一个关键特性是,在被包装的函数定义之后立即执行,这通常是在导入python模块时
# 例如以下registry.py
# 在导入registry.py模块时,被装饰的fun1() fun2()会立即执行而不需要显式调用
# 而fun3()则需要调用才会执行 因此下面的代码输出实际为:

# python3 registration.py
# running register(<function f1 at 0x100631bf8>)
# running register(<function f2 at 0x100631c80>)
# running main()
# registry -> [<function f1 at 0x100631bf8>, <function f2 at 0x100631c80>]
# running f1()
# running f2()
# running f3()


registor = []

def registry(fun):
    print('running registry{}'.format(fun))
    registor.append(fun)
    return fun

@registry
def fun1():
    print('running fun1()')

@registry
def fun2():
    print('running fun2()')

def fun3():
    print('running fun3()')

def main():
    print('running main()')
    print('registor->',registor)
    fun1()
    fun2()
    fun3()

if __name__ == '__main__':
    main()

# 上例装饰器与被装饰对象定义在同一个.py模块内 但是实际使用时 通常在一个.py模块中定义装饰器 
# 然后在另一个模块中定义被装饰对象
# 装饰器通常会在内部定义一个函数 然后将其返回以替换输入对象 而不是返回原对象

In [ ]:
'''python中的变量作用域'''
# python不要求变量声明 
# 如下函数func(a)会把输入a打印出来 并打印全局变量b

b = 6

def func(a):
    print(a)
    print(b)

func(1)

In [ ]:
# 然而观察下面的函数func2(a) 
# 安装的智能插件如Pylance 会报unbound Error 执行代码也会同样结果
'''
也许会感到困惑,但这是Python在设计上的选择:
Python假定 在函数定义体中<赋值>的变量是局部变量
因此执行到print(b)这一句时,解释器发现b没有绑定值 遂报错
'''

b = 6

def func2(a):
    print(a)
    print(b) # 然而若把此句放到b = 9之后 则不会报错 因为彼时b已经绑定值了
    b = 9

func2(1)

In [ ]:
'''闭包'''
# 闭包是指延伸了作用域的函数 其中包含函数定义体中引用 但是不在定义体中定义的非全局变量
# 函数是不是匿名的没关系 关键是函数能够访问函数定义体之外的非全局变量
# 为了方便理解 考虑如下的例子:

# 假设有个名为avg的函数,作用是计算不断增加的系列值得平均值;例如整个历史中某个商品的平均
# 收盘价,每天都要增加新价格,因此平均值要考虑至今所有的价格
# 考虑如下使用函数来实现的示例代码

def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager 

# 调用make_averager()获得avg可调用对象
avg = make_averager()
# 调用avg(new_value)开始计算平均值
av = avg(10) # series= {10},av = 10
av = avg(11) # series= {10,11},av = 10.5

# 在上面的额例子中,历史价格都存储在series内,series是make_average的局部变量,因为series是在它的定义体中初始化的
# series: seires = []
# 其次呢,我们虽然获得了一个可调用对象avg,可是在make_averager()返回之后,其本地作用域也一去不复返了,那要如何把
# 数据存储在series内呢

# 在python中,变量分为这几种
# 全局变量
# 局部变量 在本地作用域内绑定的变量
# 自由变量 在本地作用域内出现,但是未绑定的变量
# 分析上例的代码:
# def make_averager():
#  |————————————————|
#  |   series = []  | <------------ # make_averager的本地作用域 averager内出现的自由变量在此区域绑定 称为闭包
#  |————————————————|                
#     def averager(new_value):
#         series.append(new_value)  # <series>出现在本地作用域 但是未绑定 是一个自由变量
#         total = sum(series)
#         return total/len(series)
#     return averager

# averager的闭包延伸到averager的作用域之外,但是只包含自由变量series绑定的部分(并不是整个make_averager的本地作用域)
# __code__属性,编译后的函数定义体
print(avg.__code__.co_freevars) # 自由变量的名字('series')
print(avg.__code__.co_varnames) # 本地变量的名字('new_value','total')
print(avg.__closure__)          # cell对象,各个元素的名称对应于.co_freevars中的名称
print(avg.__closure__[0].cell_contents) # 第一个自由变量

# 闭包是一种函数,会保留定义函数时存在的自由变量的绑定,当调用的函数返回后,虽然作用域不存在了,仍能够使用那些绑定


In [ ]:
'''nonlocall声明'''
# 前面实现make_averager的方法效率不高,每次都把值放到数列中,然后调用sum求和
# 更实用的方法是只保留目前的总值和元素个数,如下

def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1
        total += new_value
        return total/count
    
    return averager

# 然而上面的实现方法是有问题的
# 这涉及到python中的变量名和实际对象之间的关系了
# python中,数字、字符、元组等的字面值是实际的对象,而变量名是对这些对象的引用
# 因此,当count是数字或者不可变值时, count += 1相当于 count = count + 1
# 即,在averager内部给count赋值了,这会把count变成局部变量,total也是

# 对数字、字符、元组等不可变类型来说,只能读取,不能更新,如果尝试重新绑定,如count = count + 1
# 会隐式创建局部变量count,这样count就不是自由变量,也就不会保存在闭包中

# 为了解决这个问题,python引入了nonlocal声明
# 变量前加上nonlocal,会把变量标记为自由变量,即使在函数中给函数赋值了
# 如果nonlocal声明的变量赋予了新值,闭包中保存的绑定会更新
# make_averager的正确实现如下:

def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += 1
        return total/count
    
    return averager


In [ ]:
'''带参数的装饰器'''
# python把被装饰的函数作为第一个参数传给装饰器函数,那么如何让装饰器接受其他参数呢
# 答案是,创建一个装饰器工厂函数,把参数传给它,返回一个装饰器,然后把装饰器应用到要被装饰的函数上
# 如下例

registor = []

def registry(func):
    print('running registry{}'.format(func))
    registor.append(func)
    return func

@registry
def f():
    print('running f()')

#为了便于启用或者禁用registry执行的函数注册功能,我们为它提供一个可选的active参数,
# False时不注册被装饰的函数

registor = set()

def registry2(active = True):
    def decorate(func):
        print('running registry(active={0})->decorate({1})'.format(active,func)) # active 就是自由变量
        if(active):
            registor.add(func)
        else:
            registor.discard(func)
        return func
    
    return decorate

@registry2(active=False) # 调用函数registry2() 返回装饰器decorate 用来装饰f1
def f1():
    print('runnning f1()')

@registry2()             # 调用函数registry2() 返回装饰器decorate 用来装饰f2
def f2():
    print('runnning f2()')

def f3():
    print('running f3()')

# 必须用函数调用的方式(调用装饰器工厂函数)来获得一个装饰器

In [ ]:
# @overload 装饰器
# 被装饰的函数只是用来做类型检查,最后必须跟一个没被装饰的版本,是真正被调用的
import typing

@typing.overload
def process(response : None) -> None:
    ...

@typing.overload
def process(response : int) ->tuple[int,str]:
    ...

@typing.overload
def process(response : int) ->str:
    ...

def process(response):
    pass